# SLU12- Support Vector Machines: Learning notebook

In this notebook we will be covering the following:


*  Hyperplanes
*  Maximal Margin Classifier
* Support Vector Classifier
* Support Vector Machine
* Multi-Class extension
* Support Vector Regression

New tools in this unit

* [SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
* [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)



In [1]:
import warnings
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris

warnings.simplefilter("ignore")

# load the iris dataset and train-test split it
X, y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# SVMs are not scale invariant, so we should scale our data beforehand
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Take a look at the first five observations' features
X_train[0:5, :]

array([[-0.92687146,  1.6914824 , -1.27700802, -1.20429619],
       [ 1.00327301,  0.0986066 ,  1.02610029,  1.54215304],
       [ 0.27946884, -0.12894709,  0.46436655,  0.23432007],
       [ 0.5207369 , -1.26671552,  0.68906005,  0.88823656],
       [ 1.84771122, -0.58405446,  1.30696715,  0.88823656]])

In [2]:
# Take a look at the first five observations' class
y_train[0:5]

array([0, 2, 1, 2, 2])

## Introduction

In this learning unit, we discuss the support vector machine (SVM), an approach that was developed in the computer science community in
the 1990s and that has grown in popularity since then. SVMs have been shown to perform well in a variety of settings, and are often considered one
of the best “out of the box” shallow classifiers.

![intro](media/into.png)

People often loosely refer to the maximal margin classifier, the support vector classifier, and the support vector machine as “support vector machines”. To avoid confusion, we will carefully distinguish between these three notions.

## Hyperplanes

In a p-dimensional space, a hyperplane is a flat  subspace of dimension p − 1. For instance, in two dimensions, a hyperplane is a flat one-dimensional subspace—in other words, a line. In three dimensions, a hyperplane is a flat two-dimensional subspace—that is, a plane. In p > 3
dimensions, it can be hard to visualize a hyperplane, but the notion of a (p − 1)-dimensional flat subspace still applies.

![hyperplanes](media/hyperplanes.png)

$n×p$ data matrix **X** that consists of n training observations in p-dimensional space. A hyperplane in the p-dimensional setting is defined by the below equation, 

$\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p} = 0$

Using this hyperplane, we can separate two classes of observations. For instance, all observations that satisfy

$\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p} > 0$

will be classified as one class, while all other observations that satisfy 

$\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p} < 0$

will be classified as the remaining class. 

## Maximal Margin Classifier (MMC)

The Maximal Margin Classifier is a binary classification method that makes use of the **optimal separating hyperplane**, which is the separating hyperplane that is farthest from the training observations. The minimal distance from the observations to the hyperplane is know as the **margin**. The observations that lie on the margin are known as the **support vectors**. The figure below illustrates the decision boundary (black line), the margin (dashed lines), and the support vectors (points on the dashed line) of the application of a maximal margin classifier.

![mmc](media/mmc.png)

In order to construct the optimal separating hyperplane based on a set of n training observations $x_{1}, . . . , x_{n} ∈ R_{p}$ and associated class labels $y_{1}, . . . , y_{n} $∈ {-1,1}, we must find the solution to the optimization problem

* maximize $M$

* subject to $\sum_{j=1}^{p} β^{2}=1$

* $y_{i}(\beta_{0}+ \beta_{1}x_{i1}+ ... +\beta_{p}x_{ip} =  M,    ∀ i = 1, . . . , n$) 

Here, M represents the margin, and the optimization problem chooses $\beta_{0}+ \beta_{1} + ... + \beta_{p}$ to maximize M.

The maximal margin classifier is a very natural way to perform classification, if a separating hyperplane exists. However, in many cases no separating hyperplane exists, and so there is no maximal margin classifier. In this case, the optimization problem has no solution with M >0.

![meme](media/unseparable_meme.png)

## Support Vector Classifier (SVC)

The Support Vector Classifier, also known as the soft margin classifier, is an extension of the maximal margin classifier. The soft margin allows some observations to be on the incorrect side of the margin, or even on the incorrect side of the hyperplane. 

This overcomes the MMC's limitation of not being able to handle cases where no separating hyperplane exists. In fact, even if a separating hyperplane does exist, there are instances in which a classifier based on a separating hyperplane might not be desirable. A classifier based on a separating hyperplane will necessarily perfectly classify all of the training observations, which can lead to sensitivity to individual observations.

By not perfectly separating the two classes, the SVC has greater robustness to individual observations and a better classification of *most* of the observations. The image below illustrates the decision boundary and margin of an SVC, 

![svc](media/svc.png)

As can be seen, some observations violate the hyperplane. The method to obtain this hyperplane is similar to that of the MMC, except that we now introduce some slack variables $s_{0}, ..., s_{n}$ and a non-negative tunning paramenter C.

* maximize M 

* subject to $\sum_{j=1}^{p} β^{2}=1$

* $y_{i}(\beta_{0}+ \beta_{1}x_{i1}+ ... +\beta_{p}x_{ip} ≥ M(1-s_{i})$
* $s_{i} ≥ 0$

* $\sum_{j=1}^{p} s_{i} ≤ C$

* ∀ i = 1, . . . , n


The slack variables $s_{0}, ..., s_{n}$  allow individual observations to be on the wrong side of the margin or the hyperplane. 



In [3]:
# import the SVC class
from sklearn.svm import SVC

# Create an estimator
linear_svc = SVC(kernel="linear", C=1) # don't worry about the kernel argument for now
linear_svc

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [4]:
# fit the estimator
linear_svc.fit(X_train, y_train)
# make predictions
predictions = linear_svc.predict(X_test)
predictions

array([1, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 2, 1, 2, 2, 1, 0, 2, 0, 1, 1, 2,
       2, 1, 2, 1, 1, 0, 1, 2])

In [5]:
#score the estimator
linear_svc.score(X_test, y_test)

0.9666666666666667

In [6]:
# get the support vectors
linear_svc.support_vectors_

array([[-0.92687146,  0.55371397, -1.16466128, -0.9427296 ],
       [-1.65067564, -1.72182289, -1.38935477, -1.20429619],
       [-0.32370131, -0.12894709,  0.40819318,  0.36510337],
       [ 0.15883481, -0.81160815,  0.74523342,  0.49588667],
       [ 0.5207369 , -1.26671552,  0.63288667,  0.36510337],
       [ 0.5207369 ,  0.55371397,  0.52053993,  0.49588667],
       [ 0.15883481, -0.35650077,  0.40819318,  0.36510337],
       [ 0.5207369 , -1.72182289,  0.35201981,  0.10353678],
       [-0.92687146, -1.26671552, -0.43440742, -0.15802982],
       [ 0.40010287, -1.94937658,  0.40819318,  0.36510337],
       [ 1.24454107,  0.0986066 ,  0.63288667,  0.36510337],
       [-1.16813952, -1.49426921, -0.2658873 , -0.28881311],
       [ 0.27946884, -0.35650077,  0.52053993,  0.23432007],
       [-0.56496937, -0.12894709,  0.40819318,  0.36510337],
       [ 1.12390704, -0.58405446,  0.5767133 ,  0.23432007],
       [ 1.00327301, -0.12894709,  0.68906005,  0.62666996],
       [ 0.64137092,  0.

### The C Penalty

The C penalty is the sum of all the slack variables, and it thus determines the number and severity of the violations to the margin (and the hyperplane) that the model will tolerate.  If C=0 then we are back to the MMC. As C increases, the model becomes more tolerant and thus the margin will increase. this is illustrated by the pictures below, 

![small_C](media/small_C.png)
![large_C](media/large_C.png)

The picture on the top illustrates an SVC with a small value of C, while the picture on the bottom illustrates an SVC with a large value of C. In practice, C is often chosen via cross-validation, though a value of C=1 is usually a good start.

C controls the bias-variance trade-off of the statistical learning technique. When C is small  the  classifier is highly fit to the data, which may have low bias but high variance. On the other hand, when C is larger, the margin is wider and we allow more violations to it. This results in fitting the data less hard and obtaining a classifier that is potentially more biased but with lower variance.

## Support Vector Machine (SVM)

So far we have only considered models with a linear decision boundary. Support Vector Machines are the extension of the SVC to the non-linear case. The full details of this extension are somewhat complex and beyond the scope of this learning unit, but the main ideas are detailed below. 

### Kernels

SVMs enlarge the feature space in a specific way using **kernels**. The kernel approach is simply an efficient computational approach for acomodating non-linear decision boundaries. A kernel function quantifies the similarity of two observations. For instance, to obtain the SVC we could use the following kernel,


$K(x_{i}, x_{n}) =  \sum_{j=1}^{p}x_{ij}x_{nj}$


Kernels operate in implicit feature space without ever computing the coordinates of the data in that space. In practice, they simply compute the inner products between the images of all pairs of data in the feature space. This is known as the **Kernel Trick**. 

![kernel_trick](media/kernel_trick.png)

Data not linearly separable in n-dimensional space may be linearly separable in higher dimensional space. 


One popular choice is the **polynomial kernel** of degree d, given by the equation below,

$K(x_{i}, x_{n}) = (1 + \sum_{j=1}^{p}x_{ij}x_{nj})^{d}$

![poly_kernel](media/poly_kernel.png)

Using such a kernel with d > 1, instead of the SVC, leads to a much more flexible decision boundary. It essentially amounts to fitting a support vector
classifier in a higher-dimensional space involving polynomials of degree d, rather than in the original feature space.


In [7]:
# Pass kernel='poly' and degree=d to create an
# SVM with polynomial kernel of degree d
polynomial_svm = SVC(kernel="poly", degree=3)
polynomial_svm

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='poly', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [8]:
# fit the model
polynomial_svm.fit(X_train, y_train)
# score the model
polynomial_svm.score(X_test, y_test)

0.9666666666666667

Another popular option is the **radial kernel**, 

$K(x_{i}, x_{n}) = \exp(-\gamma\sum_{j=1}^{p}(x_{ij}-x_{nj})^{2})$

Where  $\gamma$  is a positive constant. This kernel has very local behavior, meaning that the classification of new observations will be mostly determined by observations very close to it. The below picture shows how the application of a radial kernel, 

![rbf_kernel](media/rbf_kernel.png)



In [9]:
# Pass kernel='rbf' (default) to create an 
# SVM with radial kernel
radial_svm = SVC(kernel="rbf")
radial_svm

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [10]:
# fit the model
radial_svm.fit(X_train, y_train)
# score the model
radial_svm.score(X_test, y_test)

0.9666666666666667

### Multi-Class Classification

So far the estimators we have seen are only applicable to binary classification. There are ways to extend them in order to perform multi-class calssification, and two popular methods are One-Vs-one (OVO) and One-Vs-Rest (OVR)


One-Vs-One
1. Fit $\binom{K}{2}$ SVMs, each compairing a pair of classes 
2. Assign test observation to most frequently assigned class in all pairwise classifications

One-Vs-Rest
1. Fit K SVMs
2. Compare each K class to remaining K-1 classes 
2. Assign  test observation for which $\beta_{0}+ \beta_{1}x_{1}+ ... +\beta_{p}x_{p}$ is largest (this amounts to a high level of confidence that the test
observation belongs to the kth class rather than to any of the other classes) 

Luckily,  all sklearn SVM estimators already implement multi-class classification, so we don't need to do it ourselves. 

In [11]:
# You can specify which multi-class method you want your estimator to use
# through the "decision_function_shape" argument.
SVC(decision_function_shape="ovo")

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

## Support Vector Regression


The method of Support Vector Classification can be extended to solve regression problems. This method is called Support Vector Regression.

The model produced by support vector classification  depends only on a subset of the training data, because the cost function for building the model does not care about training points that lie beyond the margin. Analogously, the model produced by Support Vector Regression depends only on a subset of the training data. This is illustrated by the image below,

![SVR](media/svr.png)

The model fits a decision boundary line for which the support vectors are within a certain deviation. Then, that line is used to compute the predictions. For this, we can use the [SVR](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)  implementation from sklearn

In [12]:
# Load the Boston Dataset (Regression)
from sklearn.datasets import load_boston
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Import the SVR estimator
from sklearn.svm import SVR

# Create the SVR estimator
svr = SVR()
svr

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1,
    gamma='auto_deprecated', kernel='rbf', max_iter=-1, shrinking=True,
    tol=0.001, verbose=False)

In [13]:
# Fit the SVR estimator
svr.fit(X_train, y_train)
# Make predictions
svr.predict(X_test)

array([20.9163001 , 20.92070157, 20.99561473, 20.69537039, 20.91613525,
       20.00817319, 20.91742108, 19.94976733, 20.90735622, 20.93133462,
       21.57996022, 21.40699635, 20.92795364, 20.91609794, 20.91628788,
       21.09704682, 20.91625981, 20.91578315, 20.9162891 , 20.91885659,
       19.98594738, 20.80570061, 20.92995317, 20.91628817, 20.84617137,
       20.94494118, 20.91628871, 21.03229163, 20.91679159, 20.92010204,
       20.92493616, 20.91628827, 20.91626925, 20.91628832, 20.91974351,
       20.91628828, 20.91682316, 20.91628827, 20.62574826, 20.91438676,
       20.916304  , 20.91628605, 20.91629398, 20.90266619, 20.91628938,
       20.92853   , 20.91628827, 20.91822137, 20.91797147, 21.88809569,
       20.91631189, 20.9163249 , 20.93956001, 20.91749651, 20.91628826,
       20.12995175, 20.91628827, 20.91628806, 20.91631157, 20.9162912 ,
       20.91689957, 20.93057087, 20.91628827, 20.91626351, 21.0376725 ,
       20.91623097, 20.79308539, 20.9132977 , 20.96918893, 18.54

In [14]:
# Score the estimator (using R^2)
svr.score(X_test, y_test)

-0.05905956696924241

## Conclusion

That's it for Support Vector Machines! below is the recap of the main points of this SLU:
 

*   p-dimensional hyperplanes can be used as decision boundaries
*   Maximal Margin Classifier uses the optimal separating hyperplane
*   SVC uses a soft margin to allow misclassifications
*   SVM uses kernels to handle non-linearity
*   Extension to multi-class can be achieved with OVO and OVR
*   Extension to regression can be done with SVR

### Further readings

[The Kernel method](https://en.wikipedia.org/wiki/Kernel_method)

[Support Vector Regression](https://medium.com/coinmonks/support-vector-regression-or-svr-8eb3acf6d0ff)

 Witten, Daniela, et al. “Chapter 9.” An Introduction to Statistical Learning, by Gareth James, Springer, 2017.
